# Manejo de bases de datos para las ciencias sociales II


## Manipulando la base de datos

- Sorting
- Creación de columnas
- Casting
- Agregando datos
- Manejando valores missing


A continuación veremos cuáles son los métodos más comunes para transformar nuestra base de datos y obtener nueva información.

In [7]:
#importando pandas y leyendo nuestros datos

import pandas as pd


In [9]:
## Leyendo los archivos.
locacion_datos = "aqui  coloca el string con la locación de tus datos"
locacion_datos = "/content/Datos_abiertos_admision_2021_1_2024_1.csv" # este es el mio
df = pd.read_csv(locacion_datos)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [12]:
df.columns

Index(['IDHASH', 'COLEGIO', 'COLEGIO_DEPA', 'COLEGIO_PROV', 'COLEGIO_DIST',
       'COLEGIO_PAIS', 'COLEGIO_ANIO_EGRESO', 'ESPECIALIDAD', 'ANIO_POSTULA',
       'CICLO_POSTULA', 'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',
       'ANIO_NACIMIENTO', 'NACIMIENTO_PAIS', 'NACIMIENTO_DEPA',
       'NACIMIENTO_PROV', 'NACIMIENTO_DIST', 'SEXO', 'CALIF_FINAL', 'INGRESO',
       'MODALIDAD'],
      dtype='object')

In [ ]:
### Ordenar nuestra base de datos:


In [13]:
## una base de datos en no requiere que esté ordenada, pues la información mostrada es la misma,
# pero a veces es útil para visualizarla o para hacer operaciones.
df = df.sort_values('ANIO_NACIMIENTO', ascending = False) # por default, ascending es True.
# lo mismo, pero inplace:
#df.sort_values('ANIO_NACIMIENTO', ascending = False, inplace=True)

# ordenar por dos O más columnas:
df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'])

#df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'], ascending=[False, True])



### Creando columnas nuevas.

A continuación veremos varias maneras de crear variables nuevas, dependiendo de las circunstancias de los datos, así como nuestras necesidades/preferencias.



In [14]:
# La manera más sencilla:
# Crear una nueva columna a partir de la edad del postulante:
df['MAYOR_EDAD'] = df['ANIO_NACIMIENTO'] <= 2006
df['MENOR_EDAD'] = df['ANIO_NACIMIENTO']>= 2007

Los siguentes métodos que utilizamos para filtrar datos también sirven para crear variables:


`.loc `

Podemos crear nuevas variables ubicando en el dataframe los valores que cumplen cierta condición y reemplazando por otras.

In [15]:
# se escribe primero la fila y luego la columana
# lo que hace loc es ubicar observaciones por las etiquetas de las filas y columnas

In [16]:
df['SEXO_'] = 0
df.loc[df['SEXO'] == 'FEMENINO', 'SEXO_'] = 1

In [17]:
pd.crosstab(df['SEXO'], df['SEXO_'])

SEXO_,0,1
SEXO,,
FEMENINO,0,4802
MASCULINO,15486,0


Si queremos cambiar varias categorías a la vez, podemos utilizar  `isin()`, `replace` o `where`:

In [20]:
paises = ['ARGENTINA', 'COLOMBIA', 'VENEZUELA', 'CHILE', 'ESPAÑA',
       'ITALIA', 'EE.UU.', 'BOLIVIA', 'BRAZIL', 'JAPÓN', 'KOREA',
       'URUGUAY', 'ECUADOR',  'GUATEMALA', 'HONDURAS',
       'SUIZA', 'SIRIA', 'PARAGUAY', 'FRANCIA', 'MÉXICO'] ## df['NACIMIENTO_PAIS'].unique()

df["extranjero"] = 0
df.loc[df['NACIMIENTO_PAIS'].isin(paises), "extranjero"] = 1

In [26]:
df['OTRO_SEXO'] = df['SEXO']
df.replace({'OTRO_SEXO': {'FEMENINO': 1, 'MASCULINO': 0}}, inplace=True)

/tmp/ipython-input-4123913917.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'OTRO_SEXO': {'FEMENINO': 1, 'MASCULINO': 0}}, inplace=True)


In [ ]:
# import numpy as np
# df['extranjero'] = np.where(df['NACIMIENTO_PAIS'].isin(paises), 1, 0)


In [27]:
df.columns

Index(['IDHASH', 'COLEGIO', 'COLEGIO_DEPA', 'COLEGIO_PROV', 'COLEGIO_DIST',
       'COLEGIO_PAIS', 'COLEGIO_ANIO_EGRESO', 'ESPECIALIDAD', 'ANIO_POSTULA',
       'CICLO_POSTULA', 'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',
       'ANIO_NACIMIENTO', 'NACIMIENTO_PAIS', 'NACIMIENTO_DEPA',
       'NACIMIENTO_PROV', 'NACIMIENTO_DIST', 'SEXO', 'CALIF_FINAL', 'INGRESO',
       'MODALIDAD', 'MAYOR_EDAD', 'MENOR_EDAD', 'SEXO_', 'extranjero',
       'OTRO_SEXO'],
      dtype='object')

Utilizando `eval` podemos hacerlo con una sola expresión:


In [28]:
df['MAYOR_EDAD'] = df.eval('ANIO_NACIMIENTO <= 2006')
df['INGRESANTES_NOLIMA'] = df.eval("INGRESO == 'NO' & DOMICILIO_DEPA != 'LIMA'")
# con el "NO" estamos usando string en un comando que solo admite números



### Una forma más general: Utilizando apply y funciones lambda.

Recuerdan las funciones labmda que vimos en la clase pasada? Estas resultan muy útiles para hacer operaciones en columnas de un dataframe cuando la vectorización no es posible. A continuación veremos algunos ejemplos:

In [41]:
# El error AttributeError: 'float' object has no attribute 'title' significa que estás intentando usar el método title() (que es exclusivo de los strings) en un valor que no es un string, en este caso, un número flotante. Esto suele pasar cuando hay valores nulos (NaN) en la columna COLEGIO, ya que NaN es de tipo float. Para solucionarlo, podemos asegurarnos de que todos los valores sean strings, por ejemplo, rellenando los NaN con una cadena vacía antes de aplicar la función.
df['Colegio_'] = df['COLEGIO'].fillna('').apply(lambda x: x.title())

In [34]:
df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: x <= 2006)


In [35]:
def mayor_edad(x):
    if x <= 2006:
        return True
    else:
        return False

df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: mayor_edad(x))

También podemos crear variables categóricas en base a variables contínuas con `cut`:

In [ ]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 5, 10, 15, 20.01]

In [ ]:
df['cat_calificacion'] = pd.cut(df['CALIF_FINAL'],
                              bins = categ_bins, labels = categ_labels)

### Casting: Cambiando tipos de datos:
Hay veces cuando necetaremos cambiar el tipo de dato de una columna.    

- Por ejemplo, cuando el csv fue leido y una columna que debería ser numérica,  es leída como string.  
- Otra situación se da cuando queremos optimizar el uso de memoria de nuestra base de datos, y por ejemplo, queremos reducir el número de decimales que el sistema guarda.  
- Luego, cuando concatenamos bases de datos, los tipos entre uno y otro pueden no coincidir, y necesitaremos cambiarlos para poder hacer operaciones.


In [ ]:
df['ANIO_NACIMIENTO'] = df['ANIO_NACIMIENTO'].astype('str')

# cambiamos el tipo de datos con astype

# Cuando leemos los archivos, existe un argumento que se llama dtype, que nos permite especificar el tipo de dato de cada columna.


## Agregando datos


Para hacer agregaciones por columnas: ```axis = 0``` ó ```axis = 'index' ```
Para hacer agregaciones por filas: ```axis = 1``` ó ```axis = 'columns' ```

(dato curioso: El axis 0 en realidad se refiere a las filas, pero lo invocamos para obtener datos agregados por columnas porque es como lo mantenemos "fijo". Lo mismo para hacer operaciones por filas).

| Función     | Descripción                                               |
|-------------|-----------------------------------------------------------|
| `mean`      | Calcula el promedio de los valores.                       |
| `median`    | Encuentra la mediana de los valores.                      |
| `sum`       | Suma todos los valores.                                   |
| `min`       | Encuentra el valor mínimo.                                |
| `max`       | Encuentra el valor máximo.                                |
| `quantile`  | Calcula el cuantil especificado de los valores.           |
| `std`       | Calcula la desviación estándar de los valores.            |
| `var`       | Calcula la varianza de los valores.                       |
| `count`     | Cuenta el número de valores no nulos.                     |


In [ ]:
pbi = pd.read_csv("datos/pbi.csv")

departamentos = ['Amazonas', 'Áncash', 'Apurímac', 'Arequipa', 'Ayacucho',
       'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín',
       'La Libertad', 'Lambayeque', 'Lima', 'Prov. Const. del Callao',
       'Región Lima', 'Provincia de Lima', 'Loreto', 'Madre de Dios',
       'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes',
       'Ucayali']

In [ ]:
## Hallando la suma total de pbi departamental
pbi['total'] = pbi[departamentos].sum(axis=1)

## Hallando el promedio de pbi departamental de los últimos años:

pbi[departamentos].mean() # axis = 0 es por default. b

## Utilizando el groupby:

El groupby nos va a permitir agrupar los datos según una columna y aplicar una función a cada grupo. Es conveniente para cuando querramos obtener datos a un nivel más agregado.

In [36]:
df.columns

Index(['IDHASH', 'COLEGIO', 'COLEGIO_DEPA', 'COLEGIO_PROV', 'COLEGIO_DIST',
       'COLEGIO_PAIS', 'COLEGIO_ANIO_EGRESO', 'ESPECIALIDAD', 'ANIO_POSTULA',
       'CICLO_POSTULA', 'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',
       'ANIO_NACIMIENTO', 'NACIMIENTO_PAIS', 'NACIMIENTO_DEPA',
       'NACIMIENTO_PROV', 'NACIMIENTO_DIST', 'SEXO', 'CALIF_FINAL', 'INGRESO',
       'MODALIDAD', 'MAYOR_EDAD', 'MENOR_EDAD', 'SEXO_', 'extranjero',
       'OTRO_SEXO', 'INGRESANTES_NOLIMA', 'MAYOR_EDAD_', 'Colegio_'],
      dtype='object')

In [37]:
df.groupby('ESPECIALIDAD')['CALIF_FINAL'].mean()
#.sort_values(ascending = False)

,CALIF_FINAL
ESPECIALIDAD,
ARQUITECTURA,7.018641
CIENCIA DE LA COMPUTACIÓN,8.735210
FÍSICA,8.689300
INGENIERÍA AMBIENTAL,7.168514
INGENIERÍA CIVIL,7.579284
INGENIERÍA DE CIBERSEGURIDAD,8.371347
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL,7.612207
INGENIERÍA DE MINAS,7.448315
INGENIERÍA DE PETRÓLEO Y GAS NATURAL,6.702759


In [44]:
res = df.groupby(['ESPECIALIDAD', 'INGRESO'])['CALIF_FINAL'].mean()
## qué pasa si colocas un .reset_index() al final?

In [45]:
res.reset_index().columns

Index(['ESPECIALIDAD', 'INGRESO', 'CALIF_FINAL'], dtype='object')

###  Una forma generalizada de hacer groupbys:

In [48]:

## aquí utilizamos el método agg para aplicar varias funciones a la vez.
nueva_agregacion =df.groupby(['ESPECIALIDAD']).agg(
{'CALIF_FINAL': ['mean', 'max','min'],
 'ANIO_NACIMIENTO': ['min', 'median']}
)
nueva_agregacion.columns




MultiIndex([(    'CALIF_FINAL',   'mean'),
            (    'CALIF_FINAL',    'max'),
            (    'CALIF_FINAL',    'min'),
            ('ANIO_NACIMIENTO',    'min'),
            ('ANIO_NACIMIENTO', 'median')],
           )

In [ ]:
## los nombres de las columnas son una lista de strings, que al tener varios niveles, tendrán un multiIndex.
# En este curso no lidiaaremos con  multiIndex, así que te propongo la siguiente solución:

nueva_agregacion.columns = [''.join(col).strip() for col in nueva_agregacion.columns.values]


In [ ]:
## Otra forma de hacerlo

new_col = []
for col in nueva_agregacion.columns.values:
    a = ''.join(col).strip()
    new_col.append(a)
nueva_agregacion.columns = new_col


In [ ]:
# Otra forma de hacerlo
# df.groupby(['ESPECIALIDAD', 'INGRESO']).agg(nueva_var =  ('CALIF_FINAL', 'mean')).reset_index()

## resumiendo datos con pivot_table()
Pandas tiene un método que nos permite hacer tablas dinámicas (a lo Excel) para resumir información.



In [ ]:
pd.pivot_table(df, values = 'CALIF_FINAL', #variable con los valores que vamos a resumir
                index = 'ESPECIALIDAD', # filas
                  columns = 'INGRESO', # columnas
                    aggfunc = 'mean' # función de agregación
                    )

In [ ]:
pd.pivot_table(df,
                values = ['CALIF_FINAL', 'ANIO_NACIMIENTO'],
                index = 'ESPECIALIDAD',
                columns = 'INGRESO',
                aggfunc = {'CALIF_FINAL': ['mean', 'median'],
                            'ANIO_NACIMIENTO': 'median'})

In [ ]:
### Formatos Long-Wide:



In [ ]:
pbi.melt(id_vars="Anio") ## En caso tuviesemos más de una variable

In [ ]:
df_wide = pd.pivot_table(df, values = 'CALIF_FINAL', index = 'ESPECIALIDAD', columns = 'INGRESO', aggfunc = 'mean').reset_index()
df_wide.melt(id_vars = 'ESPECIALIDAD', value_vars = ['NO', 'SI'])

### Manejo de datos faltantes:
Los datos faltantes pueden suceder por varias razones:
-  Cuando no se necesita registrar los datos de dicha observación (por ejemplo, nivel educativo para menores de 3 años).  
-  Sin embargo, esto puede afectar nuestro análisis si son errores en el levantamiento de la información.    
    - ejemplo de variables sensibles de levantamiento.
    - errores de levantamiento de datos.
    
Su identifcación y tratamiento es importante.


In [ ]:
## IDentificando datos faltantes:
df.isna().sum() # nos da la cantidad de datos faltantes por columna.

In [ ]:
## Observando datos faltantes de un subconjunto de columnas:
df[['COLEGIO_DEPA', 'NACIMIENTO_DEPA']].isna().sum()

df = df.fillna(0) # reemplaza los datos faltantes por 0.
#
df[['COLEGIO_DEPA', 'NACIMIENTO_DEPA']].fillna('No especificado')

# Otros métodos: notna(), dropna()
